# Import CSV

In [1]:
import pandas as pd

In [2]:
fileName = 'pilots.csv'
filePath = '../input/'
data = pd.read_csv(filePath + fileName) 

# Create PDFs

In [3]:
from fpdf import FPDF
import datetime

In [4]:
title = 'Competition name'
subtitle = 'Application form'

class PDF(FPDF):
    def header(self):
        # Logo
        self.image(filePath + 'logo.png', 10, 8, 33)
        
        # TITLE
        self.set_font('Times', 'B', 20)
        # Calculate width of title and position
        w = self.get_string_width(title) + 6
        w *= 0.8
        self.set_x((210 - w) / 2)
        self.cell(w, 9, title,'C')
        
        # Line break
        self.ln(10)
        
        # SUBTITLE
        self.set_font('Times', '', 20)
        w = self.get_string_width(subtitle) + 6
        w *= 0.8
        self.set_x((210 - w) / 2)
        self.cell(0, 9,subtitle,'C')
        
        # Line break
        self.ln(40)

    def footer(self):
        # Position at 1.5 cm from bottom
        self.set_y(-15)
        # Arial italic 8
        self.set_font('Times', 'I', 8)
        # Text color in gray
        self.set_text_color(128)
        # Page number
        pdf.add_font('Reg', '', '../input/fonts/font_reg.ttf', uni=True)
        pdf.set_font('Reg', '', 8)
        self.cell(0, 10, 'Document is automatically generated with FPDF. Author: Mitja Jančič', 0, 0, 'C')

In [5]:
fontSize = 14
bigSpace = 15
midSpace = 8
smallSpace = 5
emptyPages = 20
emptyLine = '____________'
meetDirector = 'Club, organizer'
responsibility = 'Participation in this competition is at the sole responsibility of each individual pilot. The event organizer does not claim any responsibility for the pilots or other third parties.'

pdf = PDF()
pdf.add_font('Reg', '', '../input/fonts/font_reg.ttf', uni=True)
pdf.add_font('Bold', '', '../input/fonts/font_bold.ttf', uni=True)
for index, pilot in data.iterrows():
    pdf.add_page()
    
    # pilot name
    pdf.set_font('Reg', '', fontSize)
    pdf.write(fontSize, 'Pilot name: ')
    pdf.set_font('Bold', '', fontSize)
    pdf.write(fontSize, str(pilot['Full name']))
    pdf.ln(midSpace)
    
    # gender
    pdf.set_font('Reg', '', fontSize)
    pdf.write(fontSize, 'Gender: ')
    pdf.set_font('Bold', '', fontSize)
    pdf.write(fontSize, str(pilot['Sex']))
    pdf.ln(midSpace)
    
    # Licence no.
    pdf.set_font('Reg', '', fontSize)
    pdf.write(fontSize, 'Licence no.: ')
    pdf.set_font('Bold', '', fontSize)
    pdf.write(fontSize, str(pilot['Licence no.']))
    pdf.ln(midSpace)
    
    # FAI Sporting license
    pdf.set_font('Reg', '', fontSize)
    pdf.write(fontSize, 'FAI number: ')
    pdf.set_font('Bold', '', fontSize)
    pdf.write(fontSize, str(pilot['FAI number']))
    pdf.ln(midSpace)
    
    # CIVL ID
    pdf.set_font('Reg', '', fontSize)
    pdf.write(fontSize, 'CIVL ID: ')
    pdf.set_font('Bold', '', fontSize)
    pdf.write(fontSize, str(pilot['CIVL']))
    pdf.ln(midSpace)
    
    # date of birth
    pdf.set_font('Reg', '', fontSize)
    pdf.write(fontSize, 'Date of birth: ')
    pdf.set_font('Bold', '', fontSize)
    pdf.write(fontSize, str(pilot['Date of birth']))
    pdf.ln(midSpace)
    
    # Mobile phone
    pdf.set_font('Reg', '', fontSize)
    pdf.write(fontSize, 'Mobile phone: ')
    pdf.set_font('Bold', '', fontSize)
    pdf.write(fontSize, str(pilot['Mobile phone']))
    pdf.ln(midSpace)
    
    # flying since
    pdf.set_font('Reg', '', fontSize)
    pdf.write(fontSize, 'Flying since: ')
    pdf.set_font('Bold', '', fontSize)
    pdf.write(fontSize, emptyLine)
    pdf.ln(midSpace)
    
    # team
    pdf.set_font('Reg', '', fontSize)
    pdf.write(fontSize, 'Team: ')
    pdf.set_font('Bold', '', fontSize)
    pdf.write(fontSize, emptyLine + emptyLine)
    pdf.ln(midSpace)
    
    # nationality
    pdf.set_font('Reg', '', fontSize)
    pdf.write(fontSize, 'Nation: ')
    pdf.set_font('Bold', '', fontSize)
    pdf.write(fontSize, str(pilot['Country']))
    pdf.ln(midSpace)
    
    # glider
    pdf.set_font('Reg', '', fontSize)
    pdf.write(fontSize, 'Paraglider (EN 926/2): ')
    pdf.set_font('Bold', '', fontSize)
    pdf.write(fontSize, str(pilot['Manufacturer']) + ' ' + str(pilot['Glider']))
    pdf.ln(midSpace)
    
    # glider class
    pdf.set_font('Reg', '', fontSize)
    pdf.write(fontSize, 'Safety class: ')
    pdf.set_font('Bold', '', fontSize)
    pdf.write(fontSize, str(pilot['Class']))
    pdf.ln(midSpace)
    
    # glider color
    pdf.set_font('Reg', '', fontSize)
    pdf.write(fontSize, 'Paraglider color: ')
    pdf.set_font('Bold', '', fontSize)
    pdf.write(fontSize, emptyLine)
    pdf.ln(bigSpace)
    
    # health insurance
    pdf.set_font('Bold', '', fontSize)
    pdf.write(fontSize, 'Medical insurance, which includes paragliding competitions:')
    pdf.ln(midSpace)
    pdf.set_font('Reg', '', fontSize)
    pdf.write(fontSize, 'Company: ')
    pdf.write(fontSize, emptyLine + emptyLine)
    pdf.ln(midSpace)
    pdf.write(fontSize, 'Card/policy number: ')
    pdf.write(fontSize, emptyLine + emptyLine)
    pdf.ln(bigSpace)
    
    # Responsibility
    pdf.set_font('Bold', '', fontSize)
    pdf.write(fontSize, 'Responsibility:')
    pdf.ln(midSpace)
    pdf.set_font('Reg', '', fontSize)
    pdf.write(7, responsibility)
    pdf.ln(bigSpace)
    
    # Organisor
    pdf.set_font('Bold', '', fontSize)
    pdf.write(fontSize, 'Meet director: ')
    pdf.set_font('Reg', '', fontSize)
    pdf.write(fontSize, meetDirector)
    pdf.ln(midSpace)
    
    # Signature
    pdf.set_font('Bold', '', fontSize)
    pdf.write(fontSize, 'Signature by organizer: ')
    pdf.ln(bigSpace)
    
    # Date created
    pdf.set_font('Reg', '', 10)
    date = datetime.datetime.now().strftime("%I:%M:%S %p on %B %d, %Y.")
    pdf.write(fontSize, 'Application form auto-generated at ' + date)

for i in range(emptyPages):
    pdf.add_page()
    
    # pilot name
    pdf.set_font('Reg', '', fontSize)
    pdf.write(fontSize, 'Pilot name: ')
    pdf.set_font('Bold', '', fontSize)
    pdf.write(fontSize, '')
    pdf.ln(midSpace)
    
    # gender
    pdf.set_font('Reg', '', fontSize)
    pdf.write(fontSize, 'Gender: ')
    pdf.set_font('Bold', '', fontSize)
    pdf.write(fontSize, '')
    pdf.ln(midSpace)
    
    # Licence no.
    pdf.set_font('Reg', '', fontSize)
    pdf.write(fontSize, 'Licence no.: ')
    pdf.set_font('Bold', '', fontSize)
    pdf.write(fontSize, '')
    pdf.ln(midSpace)
    
    # FAI Sporting license
    pdf.set_font('Reg', '', fontSize)
    pdf.write(fontSize, 'FAI number: ')
    pdf.set_font('Bold', '', fontSize)
    pdf.write(fontSize, '')
    pdf.ln(midSpace)
    
    # CIVL ID
    pdf.set_font('Reg', '', fontSize)
    pdf.write(fontSize, 'CIVL ID: ')
    pdf.set_font('Bold', '', fontSize)
    pdf.write(fontSize, '')
    pdf.ln(midSpace)
    
    # date of birth
    pdf.set_font('Reg', '', fontSize)
    pdf.write(fontSize, 'Date of birth: ')
    pdf.set_font('Bold', '', fontSize)
    pdf.write(fontSize, '')
    pdf.ln(midSpace)
    
    # flying since
    pdf.set_font('Reg', '', fontSize)
    pdf.write(fontSize, 'Flying since: ')
    pdf.set_font('Bold', '', fontSize)
    pdf.write(fontSize, '')
    pdf.ln(midSpace)
    
    # team
    pdf.set_font('Reg', '', fontSize)
    pdf.write(fontSize, 'Team: ')
    pdf.set_font('Bold', '', fontSize)
    pdf.write(fontSize, '')
    pdf.ln(midSpace)
    
    # nationality
    pdf.set_font('Reg', '', fontSize)
    pdf.write(fontSize, 'Nation: ')
    pdf.set_font('Bold', '', fontSize)
    pdf.write(fontSize, '')
    pdf.ln(midSpace)
    
    # glider
    pdf.set_font('Reg', '', fontSize)
    pdf.write(fontSize, 'Paraglider (EN 926/2): ')
    pdf.set_font('Bold', '', fontSize)
    pdf.write(fontSize, '')
    pdf.ln(midSpace)
    
    # glider class
    pdf.set_font('Reg', '', fontSize)
    pdf.write(fontSize, 'Safety class: ')
    pdf.set_font('Bold', '', fontSize)
    pdf.write(fontSize, '')
    pdf.ln(midSpace)
    
    # glider color
    pdf.set_font('Reg', '', fontSize)
    pdf.write(fontSize, 'Paraglider color: ')
    pdf.set_font('Bold', '', fontSize)
    pdf.write(fontSize, '')
    pdf.ln(bigSpace)
    
    # health insurance
    pdf.set_font('Bold', '', fontSize)
    pdf.write(fontSize, 'Medical insurance, which includes paragliding competitions:')
    pdf.ln(midSpace)
    pdf.set_font('Reg', '', fontSize)
    pdf.write(fontSize, 'Company: ')
    pdf.write(fontSize, '')
    pdf.ln(midSpace)
    pdf.write(fontSize, 'Card/policy number: ')
    pdf.write(fontSize, '')
    pdf.ln(bigSpace)
    
    # Responsibility
    pdf.set_font('Bold', '', fontSize)
    pdf.write(fontSize, 'Responsibility:')
    pdf.ln(midSpace)
    pdf.set_font('Reg', '', fontSize)
    pdf.write(7, responsibility)
    pdf.ln(bigSpace)
    
    # Organisor
    pdf.set_font('Bold', '', fontSize)
    pdf.write(fontSize, 'Meet director: ')
    pdf.set_font('Reg', '', fontSize)
    pdf.write(fontSize, meetDirector)
    pdf.ln(midSpace)
    
    # Signature
    pdf.set_font('Bold', '', fontSize)
    pdf.write(fontSize, 'Signature by organizer: ')
    pdf.ln(bigSpace)
    
    pdf.set_font('Reg', '', 10)
    date = datetime.datetime.now().strftime("%I:%M:%S %p on %B %d, %Y.")
    pdf.write(fontSize, 'Application form auto-generated at ' + date)
    
pdf.output('../output/ApplicationForms.pdf', 'F')

''